<a href="https://colab.research.google.com/github/shekhtausif8090/First-Neural-Network/blob/master/Cats_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-07-20 10:09:45--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.188.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  34.3MB/s    in 1.9s    

2020-07-20 10:09:47 (34.3 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.2.0'

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [ ]:
zip_object.extractall("./")

In [ ]:
zip_object.close()

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [ ]:
IMG_SHAPE=(128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [24]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [27]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [28]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [29]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [30]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
16/16 [==============================] - 38s 2s/step - loss: 0.5247 - accuracy: 0.7355 - val_loss: 0.4606 - val_accuracy: 0.7940
Epoch 2/5
16/16 [==============================] - 37s 2s/step - loss: 0.4512 - accuracy: 0.7990 - val_loss: 0.4052 - val_accuracy: 0.8380
Epoch 3/5
16/16 [==============================] - 37s 2s/step - loss: 0.4015 - accuracy: 0.8340 - val_loss: 0.3604 - val_accuracy: 0.8640
Epoch 4/5
16/16 [==============================] - 37s 2s/step - loss: 0.3595 - accuracy: 0.8580 - val_loss: 0.3232 - val_accuracy: 0.8870
Epoch 5/5
16/16 [==============================] - 37s 2s/step - loss: 0.3255 - accuracy: 0.8730 - val_loss: 0.2930 - val_accuracy: 0.9050


In [31]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [32]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9049999713897705
